In [1]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300

def get_seq(path):
    path = "../data/alignments/Pol-seq/" + path 
    with open(path) as fasta_file:  # Will close handle cleanly
        for seq_record in SeqIO.parse(fasta_file, "fasta"):  # (generator)
            seq = "".join(seq_record.seq)
    return(seq)

def sliding_window(elements, window_size):
    if len(elements) <= window_size:
        return elements
    window = []
    for i in range(len(elements) - window_size + 1):
        window.append(elements[i:i+window_size])
    return(window)

def create_df(seq, window_size):
    window     = sliding_window(seq.upper(), window_size)
    P_count    = [i.count("P") for i in window]
    window_num = [i for i in range(len(window))]
    return(window, window_num, P_count)

def create_sliding_window_df(df, seq_col, taxa_col):
    window = []; window_num = []; P_count = []; seq_id = []

    for i, row in df.iterrows():
        seq = row[seq_col]    
        window_i, window_num_i, P_count_i = create_df(seq, window_size= 25)
        seq_id_i = [row[taxa_col]]*len(window_i)


        window.extend(window_i)
        window_num.extend(window_num_i)
        P_count.extend(P_count_i)
        seq_id.extend(seq_id_i)

    
    df = pd.DataFrame({"window_seq": window, "species": seq_id,
                       "window": window_num, "Proline count": P_count})
    return(df)

def create_alignment_df(align_path, align_format):
    with open(align_path) as fasta_file:  # Will close handle cleanly
        identifiers = []
        lengths = []
        seqs = []
        for seq_record in SeqIO.parse(fasta_file, align_format):  # (generator)
            identifiers.append(seq_record.description)
            lengths.append(len(seq_record.seq))
            seqs.append("".join(seq_record.seq))
        df = pd.DataFrame({"id": identifiers, "lengths": lengths, "full_seq": seqs})
    return(df)

In [2]:
# file with species and taxa assignment
align_assign = pd.read_csv("/n/groups/marks/projects/HBV_pol/data/alignments/Yingpu_March2023/seq_taxa_assignment_Yingpu.csv")

In [3]:
#read in hepadna sequence alignment
fasta_path = "/n/groups/marks/projects/HBV_pol/data/alignments/Yingpu_March2023/RT-RNaseH-aligned_yyu_clean.fa"
fasta = create_alignment_df(fasta_path, "fasta" )

In [4]:
#remove gaps
fasta['seq_nogap'] = fasta.full_seq.str.replace("-", "")
#get species name
fasta["Species"] = [i.split(" (")[0] for i in fasta.id]

In [5]:
#sliding window calculation
fasta_df = create_sliding_window_df(fasta, "seq_nogap", "Species")

In [6]:
df = fasta_df.merge(align_assign, left_on = 'species', right_on = 'Species')

In [7]:
window_corr = []
for i,row in df.iterrows():
    t = row.species
    s_max = df[df.species == t].window.max()
    diff  = df.window.max() - s_max

    window_corr.append(row["window"] + diff)
df["window_corr"] = window_corr

df = df.drop("species", axis = 1)
df = df.rename(columns = {"window": "window_left_align", "window_corr": "window_right_align"} )

In [ ]:
df.to_csv("../data/proline_sliding_window_Yingpu_align_April_19_2023.csv")